In [1]:
import csv
import sys
import statistics
from numpy import percentile

In [2]:
csv_file = open('../dataset/FilmesBrasileirosLancados.csv', 'r')
csv_reader = csv.reader(csv_file, delimiter=';')
next(csv_reader) # skip header

['"Ano de Lançamento"',
 '"Certificado de Produto Brasileiro (CPB)"',
 '"Título"',
 '"Direção"',
 '"Gênero"',
 '"Empresa Produtora Brasileira Majoritária"',
 '"UF"',
 '"Empresa Produtora Minoritária Brasileira"',
 '"UF"',
 '"Distribuidora"',
 '"Máximo de Salas"',
 '"Renda (R$)"',
 '"Público "']

## Filtrando campos necessários

In [3]:
def edit_text(rstring):
    rstring = rstring.lower()
    rstring = rstring.replace('"', '').replace("ç","c")
    rstring = rstring.replace("á", "a").replace("é", "e").replace("í","i").replace("ó","o").replace("ú","u")
    rstring = rstring.replace("ã", "a").replace("õ","o")
    rstring = rstring.replace("â", "a").replace("ê", "e").replace("î","i").replace("ô","o").replace("û","u")
    return rstring

In [4]:
data = []
for row in csv_reader:
    if row[0] != '':
        if row[11] != '"ND"' and row[12] != '"ND"': # Removing movies that don't have all information
            movie = {"Diretor": row[3].replace('"', ''),"Genero": edit_text(row[4]),"Distribuidora": row[9].replace('"', ''),"Renda": float(row[11].replace('"', '').replace(',', '.')),"Publico": int(row[12].replace('"', ''))}
            data.append(movie)

In [5]:
print(len(data))

1532


## Conversão Atributos Nominais

Diretores

In [6]:
director_lst = []
director_name = []
for r in data:
    if r['Diretor'] not in director_name:
        director_name.append(r['Diretor'])
        director_lst.append({"Diretor": r['Diretor'],"Renda": [r['Renda']],"Publico": [r['Publico']]})
    else:
        director_lst[director_name.index(r['Diretor'])]['Renda'].append(r['Renda'])
        director_lst[director_name.index(r['Diretor'])]['Publico'].append(r['Publico'])

In [7]:
for d in director_name:
    director_lst[director_name.index(d)]['Renda'] = statistics.median(director_lst[director_name.index(d)]['Renda'])
    director_lst[director_name.index(d)]['Publico'] = statistics.median(director_lst[director_name.index(d)]['Publico'])

In [8]:
def define_director(inc,pub):
    txt = ''
    if inc < 7706.96:
        txt += 'e'
    elif inc < 30227.496:
        txt += 'd'
    elif inc < 117016.172:
        txt += 'c'
    elif inc < 797518.4:
        txt += 'b'
    else:
        txt += 'a'
    if pub < 887:
        txt += 'e'
    elif pub < 3462.4:
        txt += 'd'
    elif pub < 13730.6:
        txt += 'c'
    elif pub < 93890.6:
        txt += 'b'
    else:
        txt += 'a'
    return txt

In [9]:
for r in data:
    if r['Diretor'] in director_name:
        r['Diretor'] = define_director(director_lst[director_name.index(r['Diretor'])]['Renda'],director_lst[director_name.index(r['Diretor'])]['Publico'])
    else:
        r['Diretor'] = 'Outros'

Genêro

In [10]:
genre_lst = []
for r in data:
    if r['Genero'] not in genre_lst:
        genre_lst.append(r['Genero'])

In [11]:
len(genre_lst)

4

In [12]:
for r in data:
    r['Genero'] = genre_lst.index(r['Genero'])

Distribuidora

In [13]:
csv_file2 = open('../dataset/DistribuidorasFilmesBrasil.csv', 'r')
csv_reader2 = csv.reader(csv_file2, delimiter=';')
next(csv_reader2) # skip header

['"Ano de exibição"',
 '"Distribuidora"',
 '"Número de Registro Ancine"',
 '"Origem da empresa distribuidora"',
 '"Público"',
 '"Público Filmes Brasileiros"',
 '"Renda (R$)"',
 '"Renda Filmes Brasileiros (R$)"',
 '"Títulos Exibidos"',
 '"Títulos Brasileiros  Exibidos"',
 '"Títulos Lançados"',
 '"Títulos Brasileiros Lançados"']

In [14]:
data2 = []
for row in csv_reader2:
    movie = {"Distribuidora": row[1].replace('"', ''),"Renda": float(row[6].replace('"', '').replace(',', '.')),"Lancamentos": int(row[8].replace('"', ''))}
    data2.append(movie)

In [15]:
distribuidora_lst = []
distribuidora_name = []
for r in data2:
    if r['Distribuidora'] not in distribuidora_name:
        distribuidora_name.append(r['Distribuidora'])
        distribuidora_lst.append(r)
    else:
        distribuidora_lst[distribuidora_name.index(r['Distribuidora'])]['Renda'] += r['Renda']
        distribuidora_lst[distribuidora_name.index(r['Distribuidora'])]['Lancamentos'] += r['Lancamentos']

In [16]:
onlyRenda=[]
for r in data2:
    onlyRenda.append(float(r['Renda']))

In [17]:
print('PERCENTILE',percentile(onlyRenda, [20, 40, 60, 80]))

PERCENTILE [   3947.368   31683.74   204621.278 3927832.124]


In [18]:
onlyLancamentos=[]
for r in data2:
    onlyLancamentos.append(float(r['Lancamentos']))

In [19]:
print('PERCENTILE',percentile(onlyLancamentos, [20, 40, 60, 80]))

PERCENTILE [ 1.  2.  4. 16.]


In [20]:
def define_distribu(r,l):
    txt = ''
    if r < 3947.368:
        txt = 'e'
    elif r < 31683.74:
        txt = 'd'
    elif r < 204621.278:
        txt = 'c'
    elif r < 3927832.124:
        txt = 'b'
    else:
        txt = 'a'
    if l < 423.2:
        txt += 'e'
    elif l < 2784.8:
        txt += 'd'
    elif l < 16849.8:
        txt += 'c'
    elif l < 346550.8:
        txt += 'b'
    else:
        txt += 'a'
    return txt

In [21]:
for r in data:
    if r['Distribuidora'] in distribuidora_name:
        r['Distribuidora'] = define_distribu(distribuidora_lst[distribuidora_name.index(r['Distribuidora'])]['Renda'],distribuidora_lst[distribuidora_name.index(r['Distribuidora'])]['Lancamentos'])
    else:
        r['Distribuidora'] = 0

## Exportando resultados

In [22]:
csv_out = []
for r in data:
    row = []
    row.append(r['Diretor'])
    row.append(r['Genero'])
    row.append(r['Distribuidora'])
    row.append(r['Renda'])
    row.append(r['Publico'])
    csv_out.append(row)

In [23]:
with open('../dataset/RegressionAnalysisPreProcessing.csv', 'w') as writeFile:
    writer = csv.writer(writeFile)
    writer.writerows(csv_out)